# Project 03 - Due Monday, November 19 at 1pm

*Objectives*: Use Spark to process and perform basic analysis on non-relational data, including its DataFrame and SQL interfaces.

*Grading criteria*: The tasks should all be completed, and questions should all be answered with Python code, SQL queries, shell commands, and markdown cells.  The notebook itself should be completely reproducible (using AWS EC2 instance based on the provided AMI) from start to finish; another person should be able to use the code to obtain the same results as yours.  Note that you will receive no more than partial credit if you do not add text/markdown cells explaining your thinking when appropriate.

*Attestation*: **Work in groups**.  At the end of your submitted notebook, identify the work each partner performed and attest that each contributed substantially to the work.

*Deadline*: Monday, November 19, 1pm.  One member of each group must submit your notebook to Blackboard; you should not submit it separately.

## Part 1 - Setup

Begin by setting up Spark and fetching the project data.  

**Note**: you may want to use a larger EC2 instance type than normal.  This project was prepared using a `t2.xlarge` instance.  Just remember that the larger the instance, the higher the per-hour charge, so be sure to remember to shut your instance down when you're done, as always.

### About the data

We will use JSON data from Twitter; we saw an example of this in class.  It should parse cleanly, allowing you to focus on analysis.

This data was gathered using GWU Libraries' [Social Feed Manager](http://sfm.library.gwu.edu/) application during a recent game of the MLB World Series featuring the Los Angeles Dodgers and Boston Red Sox.  This first file tells you a little bit about how it was gathered:

In [1]:
!wget -O ea26ccd641744d4a8dce84de0785186d-README.txt https://s3.amazonaws.com/2018-dmfa/project-3/ea26ccd641744d4a8dce84de0785186d-README.txt

--2018-11-19 01:26:23--  https://s3.amazonaws.com/2018-dmfa/project-3/ea26ccd641744d4a8dce84de0785186d-README.txt
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.132.245
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.132.245|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1145 (1.1K) [text/plain]
Saving to: ‘ea26ccd641744d4a8dce84de0785186d-README.txt’

ea26ccd641744d4a8dc 100%[===================>]   1.12K  --.-KB/s    in 0s      

2018-11-19 01:26:23 (92.9 MB/s) - ‘ea26ccd641744d4a8dce84de0785186d-README.txt’ saved [1145/1145]



In [2]:
!cat ea26ccd641744d4a8dce84de0785186d-README.txt

This is an export created with Social Feed Manager.

EXPORT INFORMATION
Selected seeds: All seeds
Export id: ea26ccd641744d4a8dce84de0785186d
Export type: twitter_filter
Format: Full JSON
Export completed:  Oct. 30, 2018, 9:45:59 a.m. EDT
Deduplicate: Yes

COLLECTION INFORMATION
Collection name: 2018-world-series

Collection id: 4e2564448b144915b6a0eb1899075a44
Collection set: 2018-mlb-playoffs (collection set id 5a00efa0bddf4be2aa19c6df9788ff6e)
Harvest type: Twitter filter

Harvest options:

Seeds:
* Track: dodgers,redsox,red sox,bossox,world series,RedSoxVsDodgers,ladodgers,bostonredsox - Active

Change log:

Change to 2018-world-series (collection) on Oct. 30, 2018, 8:16:14 a.m. EDT by dchud:
Note: Series ended Sunday night after five games, waited ~36 hours.

Change to 2018-world-series (collection) on Oct. 23, 2018, 6:01:28 p.m. EDT by dchud:

Change to Track: dodgers,redsox,red sox,bossox,world series,RedSoxVsDodgers,ladodgers,bostonredsox (seed) on Oct. 23, 2018, 6:01:18 p.m. E

The most important pieces in that metadata are:

 * It tracked tweets that mentioned "dodgers" or "redsox" and several additional related terms.  Every item in this set should refer to one or more of these terms.
 * This data was deduplicated; we should not see individual tweets more than once.
 * Data was collected between October 23 and October 30.  All five games of the Series were played during this time.
 
You should not need to know anything about baseball to complete this assignment, but if you have baseball questions (or Twitter questions!) please ask on the discussion forum.

**Please note**: sometimes social media data contains offensive material.  This data set has not been filtered; if you do come across something inappropriate, please do your best to ignore it if you can.

## Fetch the data

The following files are available:

 * https://s3.amazonaws.com/2018-dmfa/project-3/ea26ccd641744d4a8dce84de0785186d_009.json
 * https://s3.amazonaws.com/2018-dmfa/project-3/ea26ccd641744d4a8dce84de0785186d_010.json
 * https://s3.amazonaws.com/2018-dmfa/project-3/ea26ccd641744d4a8dce84de0785186d_011.json
 * https://s3.amazonaws.com/2018-dmfa/project-3/ea26ccd641744d4a8dce84de0785186d_012.json
 
### Q1.1 - Select at least one and obtain it using `wget`.  Verify the file sizes using the command line.

Each file should contain exactly 100,000 tweets.  

*Note*: you are only required to use one of these files, but you may use more than one.  It will be easier to process more data if you use a larger EC2 instance type, as suggested above.  Use the exact same set of files throughout the assignment.

**Answer**

In [3]:
!wget -O ea26ccd641744d4a8dce84de0785186d_009.json https://s3.amazonaws.com/2018-dmfa/project-3/ea26ccd641744d4a8dce84de0785186d_009.json

--2018-11-19 01:26:23--  https://s3.amazonaws.com/2018-dmfa/project-3/ea26ccd641744d4a8dce84de0785186d_009.json
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.132.245
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.132.245|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 554277526 (529M) [application/json]
Saving to: ‘ea26ccd641744d4a8dce84de0785186d_009.json’

ea26ccd641744d4a8dc 100%[===================>] 528.60M  54.8MB/s    in 9.1s    

2018-11-19 01:26:32 (58.4 MB/s) - ‘ea26ccd641744d4a8dce84de0785186d_009.json’ saved [554277526/554277526]



For your reference, here is the text of one Tweet, randomly selected from one of these files.  You might wish to study its structure and refer to it later.

In [4]:
!cat *.json | shuf -n 1 > example-tweet.json

In [5]:
import json
print(json.dumps(json.load(open("example-tweet.json")), indent=2))

{
  "user": {
    "protected": false,
    "is_translator": false,
    "following": null,
    "contributors_enabled": false,
    "statuses_count": 564,
    "profile_sidebar_border_color": "C3EBE8",
    "url": null,
    "description": null,
    "profile_background_tile": false,
    "screen_name": "cliftonalston",
    "id": 268332482,
    "default_profile_image": false,
    "notifications": null,
    "geo_enabled": true,
    "followers_count": 41,
    "time_zone": null,
    "lang": "en",
    "profile_text_color": "333333",
    "verified": false,
    "profile_banner_url": "https://pbs.twimg.com/profile_banners/268332482/1464925536",
    "friends_count": 323,
    "location": "Los Angeles, CA",
    "listed_count": 0,
    "default_profile": false,
    "profile_background_color": "C1C9EB",
    "utc_offset": null,
    "id_str": "268332482",
    "name": "Cliff Alston",
    "profile_link_color": "1B95E0",
    "translator_type": "none",
    "profile_sidebar_fill_color": "DCF5E4",
    "profile_imag

You can find several key elements in this example; the text, time, and language of the tweet, whether it was a reply to another user, the user's screen name along with their primary language and other account information like creation date, follower/friend/tweet counts, and perhaps their location.  If there are hashtags, user mentions, or urls present in their tweet, they will be present in the `entities` section; these are not present in every tweet.  If this is a retweet, you will see the original tweet and its information nested within.

### Q1.2 - Start up Spark, and verify the file sizes.

We will use our normal startup sequence here:

In [6]:
import findspark

In [7]:
findspark.init()

In [8]:
from pyspark import SparkContext

In [9]:
spark = SparkContext(appName='project-03')

In [10]:
spark

<SparkContext master=local[*] appName=project-03>

In [11]:
from pyspark import SQLContext

In [12]:
sqlc = SQLContext(spark)

In [13]:
sqlc

In [14]:
tweets = sqlc.read.json("ea26ccd6*.json")

Verify that Spark has loaded the same number of tweets you saw before:

**Answer**

In [15]:
tweets.rdd.count()

100000

## Part 2 - Comparing DataFrames and Spark SQL

For the next three questions, we will look at operations using both DataFrames and SQL queries. Note that `tweets` is already a DataFrame:

In [16]:
tweets

DataFrame[contributors: string, coordinates: struct<coordinates:array<double>,type:string>, created_at: string, display_text_range: array<bigint>, entities: struct<hashtags:array<struct<indices:array<bigint>,text:string>>,media:array<struct<additional_media_info:struct<description:string,embeddable:boolean,monetizable:boolean,title:string>,display_url:string,expanded_url:string,id:bigint,id_str:string,indices:array<bigint>,media_url:string,media_url_https:string,sizes:struct<large:struct<h:bigint,resize:string,w:bigint>,medium:struct<h:bigint,resize:string,w:bigint>,small:struct<h:bigint,resize:string,w:bigint>,thumb:struct<h:bigint,resize:string,w:bigint>>,source_status_id:bigint,source_status_id_str:string,source_user_id:bigint,source_user_id_str:string,type:string,url:string>>,symbols:array<struct<indices:array<bigint>,text:string>>,urls:array<struct<display_url:string,expanded_url:string,indices:array<bigint>,url:string>>,user_mentions:array<struct<id:bigint,id_str:string,indices:a

To issue SQL queries, we need to register a table based on `tweets`:

In [17]:
tweets.createOrReplaceTempView("tweets")

### Q2.1 - Which 10 languages are most commonly used in tweets?  Verify your result by executing it with both the dataframe and with SQL.

Hint: for the dataframe, use `groupBy`, `count`, and `orderBy`.  See the documentation at https://spark.apache.org/docs/2.3.1/api/python/pyspark.sql.html for details on these and other functions.

**Answer**

**Find the 10 most common languages used in tweets:**

***Using dataframe***

In [18]:
tweets.groupBy("lang") \
    .count() \
    .orderBy("count", ascending = False) \
    .show(10)

+----+-----+
|lang|count|
+----+-----+
|  en|88545|
| und| 5217|
|  es| 4927|
|  pt|  402|
|  fr|  266|
|  ht|  136|
|  ja|   98|
|  tl|   86|
|  in|   55|
|  ca|   41|
+----+-----+
only showing top 10 rows



***Using SQL***

In [19]:
sqlc.sql("""
    SELECT lang, COUNT(*) AS count
    FROM tweets
    GROUP BY lang
    ORDER BY count DESC
""").show(10)

+----+-----+
|lang|count|
+----+-----+
|  en|88545|
| und| 5217|
|  es| 4927|
|  pt|  402|
|  fr|  266|
|  ht|  136|
|  ja|   98|
|  tl|   86|
|  in|   55|
|  ca|   41|
+----+-----+
only showing top 10 rows



### Q2.2 - Which 10 time zones are most common among users?  Verify your result with both the dataframe and SQL.

*Note*: for this question, you may leave NULL values present in your results, as a way to help you understand what data is present and what is missing.

**Answer**

**Find the 10 most common time zones among users:**

***Using dataframe***

In [20]:
tweets.groupBy("user.time_zone") \
    .count() \
    .orderBy("count", ascending = False) \
    .show(10)

+---------+------+
|time_zone| count|
+---------+------+
|     null|100000|
+---------+------+



**Check the percentage of tweets from verified users:**

In [21]:
tweets.filter("user.verified = 'true'").count() / tweets.count()

0.01998

***Using SQL***

In [22]:
sqlc.sql("""
    SELECT user.time_zone, COUNT(*) AS count
    FROM tweets
    GROUP BY user.time_zone
    ORDER BY count DESC
""").show(10)

+---------+------+
|time_zone| count|
+---------+------+
|     null|100000|
+---------+------+



**Check the percentage of tweets from verified users:**

In [23]:
sqlc.sql("""
    SELECT COUNT(*) / (SELECT COUNT(*) FROM tweets) AS percentile
    FROM tweets
    WHERE user.verified = 'true'
""").show(10)

+----------+
|percentile|
+----------+
|   0.01998|
+----------+



### Q2.3 - How many tweets mention the Dodgers?  How many mention the Red Sox?  How many mention both?

You may use either the dataframe or SQL to answer.  Explain why you have chosen that approach.

Hint:  you will want to look at the value of the `text` field.

**Answer**

I used both dataframe and SQL to answer the requested questions. To be honest, I prefer the SQL method because the logic of these queries is more straightforward. Other people, including future me, can understand what the query is doing by just reading it. So, we expect that SQL will be easier to re-read at a later date even if we haven't looked at the code recently.

**Check the total number of tweets mentioning the Dodgers/Red Sox/Both:**

***Using dataframe***

The number of tweets mention the Dodgers:

In [24]:
tweets.filter("LOWER(text) LIKE '%dodgers%'") \
    .count()

24555

The number of tweets mention the Red Sox:

In [25]:
tweets.filter("(LOWER(text) LIKE '%redsox%') OR (LOWER(text) LIKE '%red sox%')") \
    .count()

56951

The number of tweets mention both the Dodgers and the Red Sox:

In [26]:
tweets.filter("LOWER(text) LIKE '%dodgers%' AND ((LOWER(text) LIKE '%redsox%') OR (LOWER(text) LIKE '%red sox%'))") \
    .count()

5582

***Using SQL***

The number of tweets mention the Dodgers:

In [27]:
sqlc.sql("""
    SELECT COUNT(*) AS dodgers_tweets_count
    FROM tweets
    WHERE LOWER(text) LIKE '%dodgers%'
""").show()

+--------------------+
|dodgers_tweets_count|
+--------------------+
|               24555|
+--------------------+



The number of tweets mention the Red Sox:

In [28]:
sqlc.sql("""
    SELECT COUNT(*) AS redsox_tweets_count
    FROM tweets
    WHERE LOWER(text) LIKE '%redsox%' OR LOWER(text) LIKE '%red sox%'
""").show()

+-------------------+
|redsox_tweets_count|
+-------------------+
|              56951|
+-------------------+



The number of tweets mention both the Dodgers and the Red Sox:

In [29]:
sqlc.sql("""
    SELECT COUNT(*) AS dodgers_redsox_tweets_count
    FROM tweets
    WHERE LOWER(text) LIKE '%dodgers%'
      AND (LOWER(text) LIKE '%redsox%' OR LOWER(text) LIKE '%red sox%')
""").show()

+---------------------------+
|dodgers_redsox_tweets_count|
+---------------------------+
|                       5582|
+---------------------------+



## Part 3 - More complex queries

For this section, you may choose to use dataframe queries or SQL.  If you wish, you may verify results by using both, as in Part 2, but this is not required for this section.

### Q3.1 - Team mentions by location

In which users' locations are the Red Sox and the Dodgers being mentioned the most?  Consider each team separately, one at a time.  Discuss your findings.

Hint:  you may use either the time zones or user-specified locations for this question.

**Answer**

**Find the users' locations where Dodgers/Red Sox is mentioned the most:**

***Using dataframe***

Find the users' locations where Dodgers is mentioned the most:

In [30]:
tweets.select("user.location", "text") \
    .filter("LOWER(text) LIKE '%dodgers%'") \
    .filter("location IS NOT NULL") \
    .groupBy("location") \
    .count() \
    .orderBy("count", ascending = False) \
    .show(10)

+-----------------+-----+
|         location|count|
+-----------------+-----+
|  Los Angeles, CA|  765|
|  California, USA|  373|
|    United States|  272|
|      Los Angeles|  255|
|       Boston, MA|  221|
|      Houston, TX|  119|
|Dhaka, Bangladesh|  114|
|           México|  112|
|       California|  103|
|    Las Vegas, NV|   99|
+-----------------+-----+
only showing top 10 rows



Find the users' locations where Red Sox is mentioned the most:

In [31]:
tweets.select("user.location", "text") \
    .filter("(LOWER(text) LIKE '%redsox%') OR (LOWER(text) LIKE '%red sox%')") \
    .filter("location IS NOT NULL") \
    .groupBy("location") \
    .count() \
    .orderBy("count", ascending = False) \
    .show(10)

+------------------+-----+
|          location|count|
+------------------+-----+
|        Boston, MA| 1808|
|Massachusetts, USA|  714|
|     United States|  648|
|            Boston|  429|
|   Los Angeles, CA|  325|
|     Massachusetts|  257|
|       Chicago, IL|  235|
|    Washington, DC|  214|
|               USA|  209|
|   California, USA|  201|
+------------------+-----+
only showing top 10 rows



***Using SQL***

Find the users' locations where Dodgers is mentioned the most:

In [32]:
sqlc.sql("""
    SELECT user.location, COUNT(*) AS count
    FROM tweets
    WHERE LOWER(text) LIKE '%dodgers%'
      AND user.location IS NOT NULL
    GROUP BY user.location
    ORDER BY count DESC
""").show(10)

+-----------------+-----+
|         location|count|
+-----------------+-----+
|  Los Angeles, CA|  765|
|  California, USA|  373|
|    United States|  272|
|      Los Angeles|  255|
|       Boston, MA|  221|
|      Houston, TX|  119|
|Dhaka, Bangladesh|  114|
|           México|  112|
|       California|  103|
|    Las Vegas, NV|   99|
+-----------------+-----+
only showing top 10 rows



Find the users' locations where Red Sox is mentioned the most:

In [33]:
sqlc.sql("""
    SELECT user.location, COUNT(*) AS count
    FROM tweets
    WHERE (LOWER(text) LIKE '%redsox%' OR LOWER(text) LIKE '%red sox%')
      AND user.location IS NOT NULL
    GROUP BY user.location
    ORDER BY count DESC
""").show(10)

+------------------+-----+
|          location|count|
+------------------+-----+
|        Boston, MA| 1808|
|Massachusetts, USA|  714|
|     United States|  648|
|            Boston|  429|
|   Los Angeles, CA|  325|
|     Massachusetts|  257|
|       Chicago, IL|  235|
|    Washington, DC|  214|
|               USA|  209|
|   California, USA|  201|
+------------------+-----+
only showing top 10 rows



**Discussion**

For the Dodgers fans, we can see that 3 of the Top 4 locations are composed of variations of the names for Los Angeles and California with the remaining spot in the Top 4 simply referencing the United States as a whole. The Top 10 is also composed of locations for Mexico and Las Vegas, which we expect to be geographical areas with a Dodgers fan base. The Top 10 list also shows Dhaka, Bangladesh as a surprise appearance, so the Dodgers may also have a strong fan base in this region too.

For the Red Sox fans, we also see variations of the name for Boston and Massachusetts within 3 of the Top 4 spots. Once again, we also see the remaining spot in the Top 4 referencing the United States as a whole. For locations outside of the Massachusetts area, Washington DC and Chicago also made the Top 10 list, so the Red Sox may have a strong fan base in these metropolitan areas. Overall, we see a higher number for the most frequent location (1,808) for the Red Sox fans when compared to the Dodgers fans (765).

### Q3.2 - Which Twitter users are being replied to the most?

Discuss your findings.

Hint: use the top-level `in_reply_to_screen_name` for this.

**Answer**

**Find the top 10 Twitter users being replied to the most:**

***Using dataframe***

In [34]:
tweets.select("in_reply_to_screen_name") \
    .filter("in_reply_to_screen_name IS NOT NULL") \
    .groupBy("in_reply_to_screen_name") \
    .count() \
    .orderBy("count", ascending = False) \
    .show(10)

+-----------------------+-----+
|in_reply_to_screen_name|count|
+-----------------------+-----+
|                 RedSox| 1360|
|                Dodgers|  814|
|                    MLB|  163|
|          DodgersNation|  101|
|         DonnieWahlberg|   77|
|           DAVIDprice24|   75|
|                Rangers|   59|
|             EJerezESPN|   52|
|        MLBStatoftheDay|   43|
|             VeniceMase|   42|
+-----------------------+-----+
only showing top 10 rows



***Using SQL***

In [35]:
sqlc.sql("""
    SELECT in_reply_to_screen_name, COUNT(*) AS count
    FROM tweets
    WHERE in_reply_to_screen_name IS NOT NULL
    GROUP BY in_reply_to_screen_name
    ORDER BY count DESC
""").show(10)

+-----------------------+-----+
|in_reply_to_screen_name|count|
+-----------------------+-----+
|                 RedSox| 1360|
|                Dodgers|  814|
|                    MLB|  163|
|          DodgersNation|  101|
|         DonnieWahlberg|   77|
|           DAVIDprice24|   75|
|                Rangers|   59|
|             EJerezESPN|   52|
|        MLBStatoftheDay|   43|
|             VeniceMase|   42|
+-----------------------+-----+
only showing top 10 rows



**Discussion**

For the Twitter users with the most replies, we see that the official accounts for the Red Sex, Dodgers, and Major League Baseball are at the top of the list. For the rest of the Top 10 list, we see that the accounts for DodgersNation, EJerezESPN, and VeniceMase pertain to the Dodgers fan base. We also noticed that the DonnieWahlberg and DAVIDprice24 accounts pertain to Red Sox fans because the accounts represent an actor from Boston and one of the team's starting pitchers, respectively.

### Q3.3 - Which 10 verified users have the most followers?  Which 10 unverified users have the most followers?

Provide both the screen names and follower counts for each.

Discuss your findings.

**Answer**

**Find the top 10 verified/unverified users having the most followers:**

***Using dataframe***

For verified users:

In [36]:
from pyspark.sql.functions import max
tweets.select("user.screen_name", "user.followers_count") \
    .filter("user.verified = 'true'") \
    .groupBy("screen_name") \
    .agg(max("followers_count")) \
    .orderBy("max(followers_count)", ascending = False) \
    .show(10)

+---------------+--------------------+
|    screen_name|max(followers_count)|
+---------------+--------------------+
|            MLB|             8296241|
|    lopezdoriga|             7678758|
|El_Universal_Mx|             4836985|
|   MagicJohnson|             4685432|
|        Milenio|             4165323|
|    MarketWatch|             3611461|
| TheRealStanLee|             3340350|
|       Newsweek|             3324621|
|  NateSilver538|             3156640|
|          Migos|             2556415|
+---------------+--------------------+
only showing top 10 rows



Much higher number of verified users...

a few user names, such as magic johnson, stick out...

For unverified users:

In [37]:
from pyspark.sql.functions import max
tweets.select("user.screen_name", "user.followers_count") \
    .filter("user.verified = 'false'") \
    .groupBy("screen_name") \
    .agg(max("followers_count")) \
    .orderBy("max(followers_count)", ascending = False) \
    .show(10)

+---------------+--------------------+
|    screen_name|max(followers_count)|
+---------------+--------------------+
|   DRJAMESCABOT|             2185433|
|       PAMsLOvE|              688117|
| mlbtraderumors|              666915|
| Miguel_Gurwitz|              568885|
|        ruleiro|              520622|
| RealKentMurphy|              443846|
|thebrittanyxoxo|              325224|
|FakeSportsCentr|              318999|
|    CelticsLife|              296296|
|    milenagimon|              226625|
+---------------+--------------------+
only showing top 10 rows



***Using SQL***

For verified users:

In [38]:
sqlc.sql("""
    SELECT user.screen_name AS screen_name, MAX(user.followers_count) AS follower_counts
    FROM tweets
    WHERE user.verified = 'true'
    GROUP BY screen_name
    ORDER BY follower_counts DESC
""").show(10)

+---------------+---------------+
|    screen_name|follower_counts|
+---------------+---------------+
|            MLB|        8296241|
|    lopezdoriga|        7678758|
|El_Universal_Mx|        4836985|
|   MagicJohnson|        4685432|
|        Milenio|        4165323|
|    MarketWatch|        3611461|
| TheRealStanLee|        3340350|
|       Newsweek|        3324621|
|  NateSilver538|        3156640|
|          Migos|        2556415|
+---------------+---------------+
only showing top 10 rows



For unverified users:

In [39]:
sqlc.sql("""
    SELECT user.screen_name AS screen_name, MAX(user.followers_count) AS follower_counts
    FROM tweets
    WHERE user.verified = 'false'
    GROUP BY screen_name
    ORDER BY follower_counts DESC
""").show(10)

+---------------+---------------+
|    screen_name|follower_counts|
+---------------+---------------+
|   DRJAMESCABOT|        2185433|
|       PAMsLOvE|         688117|
| mlbtraderumors|         666915|
| Miguel_Gurwitz|         568885|
|        ruleiro|         520622|
| RealKentMurphy|         443846|
|thebrittanyxoxo|         325224|
|FakeSportsCentr|         318999|
|    CelticsLife|         296296|
|    milenagimon|         226625|
+---------------+---------------+
only showing top 10 rows



**Discussion**

For our analysis of the verified versus unverified users, we identified that the verified users have a much higher number of followers when compared to the unverified users. Even though a few of the verified accounts do not appear related to baseball, we still see that the verified account with the lowest number of followers on the Top 10 list actually has more followers than unverified with the highest number of followers.

For the verified users, we noticed a few household names, such as MagicJohnson, who is a former player for the LA Lakers and a partial-owner of the LA Dodgers. For the unverified users, we noticed that a few of the most popular accounts, such as mlbtraderumors, FakeSportsCentr, and CelticsLife, are related to sports and were likely tweeting about the World Series.  

### Q3.4 - What are the most popular sets of hashtags among users with many followers?  Are they the same as among users with few followers?

Decide for yourself exactly how many followers you believe to be "many", and explain your decision.  You may use queries and statistics to support this decision if you wish.

Hint: if your sample tweet above does not include hashtags under the `entities` field, generate a new example by running the `shuf` command again until you find one that does.

Hint 2: the hashtag texts will be in an array, so you may need some functions you haven't used before.  If you're using SQL, see the docs for [Hive SQL](https://docs.treasuredata.com/articles/hive-functions) for details, (and consider `CONCAT_WS`, for example).

Discuss your findings.

**Answer**

**Explore the data**

Check the basic statistics of followers_count:

In [40]:
tweets.describe("user.followers_count").show()

+-------+----------------+
|summary| followers_count|
+-------+----------------+
|  count|          100000|
|   mean|      4104.94204|
| stddev|85071.9871101182|
|    min|               0|
|    max|         8296241|
+-------+----------------+



Check the quantitles of followers_count:

In [41]:
tweets.select("user.followers_count") \
    .approxQuantile("followers_count", [0.01, 0.25, 0.50, 0.75, 0.99], 0.00)

[2.0, 141.0, 346.0, 803.0, 35749.0]

We decided to consider the top quartile as the Twitter users with 'many' followers. This defines the 'many' category as Twitter users with more than 803 followers. 

We also consider the bottom quartile as the Twitter users with 'few' followers. This defines the 'few' category as Twitter users with less than 141 followers. 

Check the total number of tweets having hashtags, where the users' followers_count is larger than Q3:

In [42]:
tweets.filter("user.followers_count > 803 AND SIZE(entities.hashtags.text) > 0") \
    .count()

9522

Check the total number of tweets having hashtags, where the users' followers_count is less than Q1:

In [43]:
tweets.filter("user.followers_count < 141 AND SIZE(entities.hashtags.text) > 0") \
    .count()

9607

**Find the most popular sets of hashtags**

***Using dataframe***

Among users with many followers:

In [44]:
tweets.filter("user.followers_count > 803 AND SIZE(entities.hashtags.text) > 0") \
    .groupBy("entities.hashtags.text") \
    .count() \
    .orderBy("count", ascending = False) \
    .show(10)

+--------------------+-----+
|                text|count|
+--------------------+-----+
|            [RedSox]| 1228|
|       [WorldSeries]|  969|
|[RedSox, WorldSer...|  561|
|        [DAMAGEDONE]|  437|
|           [Dodgers]|  421|
|           [SALEDAY]|  272|
|          [DoDamage]|  272|
|[REDSOX, DAMAGEDONE]|  267|
|[WorldSeries, DoD...|  259|
|[WorldSeries, Red...|  175|
+--------------------+-----+
only showing top 10 rows



Among users with few followers:

In [45]:
tweets.filter("user.followers_count < 141 AND SIZE(entities.hashtags.text) > 0") \
    .groupBy("entities.hashtags.text") \
    .count() \
    .orderBy("count", ascending = False) \
    .show(10)

+--------------------+-----+
|                text|count|
+--------------------+-----+
|       [WorldSeries]| 1112|
|            [RedSox]|  690|
|        [DAMAGEDONE]|  684|
|[REDSOX, DAMAGEDONE]|  499|
|             [HR4HR]|  429|
|           [Dodgers]|  375|
|          [DoDamage]|  366|
|[WorldSeries, DoD...|  357|
|[RedSox, WorldSer...|  344|
|           [SALEDAY]|  233|
+--------------------+-----+
only showing top 10 rows



***Using SQL***

Among users with many followers:

In [46]:
sqlc.sql("""
    SELECT entities.hashtags.text AS hashtags_set, COUNT(*) AS count
    FROM tweets
    WHERE user.followers_count > 803
      AND SIZE(entities.hashtags.text) > 0
    GROUP BY hashtags_set
    ORDER BY count DESC
""").show(10)

+--------------------+-----+
|        hashtags_set|count|
+--------------------+-----+
|            [RedSox]| 1228|
|       [WorldSeries]|  969|
|[RedSox, WorldSer...|  561|
|        [DAMAGEDONE]|  437|
|           [Dodgers]|  421|
|           [SALEDAY]|  272|
|          [DoDamage]|  272|
|[REDSOX, DAMAGEDONE]|  267|
|[WorldSeries, DoD...|  259|
|[WorldSeries, Red...|  175|
+--------------------+-----+
only showing top 10 rows



Among users with few followers:

In [47]:
sqlc.sql("""
    SELECT entities.hashtags.text AS hashtags_set, COUNT(*) AS count
    FROM tweets
    WHERE user.followers_count < 141
      AND SIZE(entities.hashtags.text) > 0
    GROUP BY hashtags_set
    ORDER BY count DESC
""").show(10)

+--------------------+-----+
|        hashtags_set|count|
+--------------------+-----+
|       [WorldSeries]| 1112|
|            [RedSox]|  690|
|        [DAMAGEDONE]|  684|
|[REDSOX, DAMAGEDONE]|  499|
|             [HR4HR]|  429|
|           [Dodgers]|  375|
|          [DoDamage]|  366|
|[WorldSeries, DoD...|  357|
|[RedSox, WorldSer...|  344|
|           [SALEDAY]|  233|
+--------------------+-----+
only showing top 10 rows



**Discussion**

We analyzed the hashtags used by Twitter users with 'many' and 'few' followers and found similar patterns regardless of the defined categories. For example, the WorldSeries, RedSox, and DAMAGEDONE hashtags are the most frequently used hashtags for both sets of user groups. Also, we notice hashtags for the Dodgers and SALEDAY, referencing one of the Red Sox pitchers, as popular hashtags for both sets of groups. 

We also identified that the Red Sox were referenced more in the hashtags than the Dodgers. We expect this behavior is explained by the Red Sox winning the World Series in only 5 games.

### Q3.5 - Analyze common words in tweet text

Following the example in class, use `tweets.rdd` to find the most common interesting words in tweet text.  To keep it "interesting", add a filter that removes at least 10 common stop words found in tweets, like "a", "an", "the", and "RT" (you might want to derive these stop words from initial results).  To split lines into words, a simple split on text whitespace like we had in class is sufficient; you do not have to account for punctuation.

After you find the most common words, use dataframe or SQL queries to find patterns among how those words are used.  For example, are they more frequently used by Dodgers or Red Sox fans, or by people in one part of the country over another?  Explore and see what you can find, and discuss your findings.

Hint: don't forget all the word count pipeline steps we used earlier in class.

**Answer**

**Find the most common words used in tweet texts (without any filter or transformation):**

In [48]:
tweets.rdd.flatMap(lambda r: r['text'].lower().split(' ')) \
    .map(lambda t: (t, 1)) \
    .reduceByKey(lambda a, b: a + b) \
    .takeOrdered(10, key=lambda pair: -pair[1])

[('the', 63223),
 ('rt', 53865),
 ('a', 24728),
 ('to', 23436),
 ('world', 23401),
 ('series', 20265),
 ('red', 18554),
 ('sox', 16619),
 ('in', 16138),
 ('@redsox:', 14766)]

**Get the txt file containing common English stop-words from the Internet:**

In [49]:
!wget -O stopwords.txt https://www.textfixer.com/tutorials/common-english-words.txt

--2018-11-19 01:28:04--  https://www.textfixer.com/tutorials/common-english-words.txt
Resolving www.textfixer.com (www.textfixer.com)... 216.172.105.107
Connecting to www.textfixer.com (www.textfixer.com)|216.172.105.107|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 551 [text/plain]
Saving to: ‘stopwords.txt’

stopwords.txt       100%[===================>]     551  --.-KB/s    in 0s      

2018-11-19 01:28:04 (134 MB/s) - ‘stopwords.txt’ saved [551/551]



**Read the stop-words file in a list and append the word 'rt' we find being common above:**

In [50]:
with open("stopwords.txt", "r") as f:
    stopwords = [w for line in f for w in line.split(',')]
stopwords.append('rt') 

**Find the most common words used in tweet texts again (in this case, remove stop-words we get above, as well as punctuations):**

In [51]:
import re
common_words = tweets.rdd.flatMap(lambda r: r['text'].lower().split(' ')) \
    .map(lambda t: re.sub(r'[^a-z0-9]', '', t)) \
    .filter(lambda t: t not in stopwords) \
    .filter(lambda t: len(t) > 0) \
    .map(lambda t: (t, 1)) \
    .reduceByKey(lambda a, b: a + b) \
    .takeOrdered(10, key = lambda pair: -pair[1])
common_words

[('redsox', 39905),
 ('dodgers', 24456),
 ('series', 24254),
 ('world', 23631),
 ('sox', 19968),
 ('red', 18769),
 ('worldseries', 13045),
 ('boston', 8379),
 ('go', 6778),
 ('win', 6734)]

**Get the list of the 10 most common words used in tweet texts:**

In [52]:
common_words_text = [w[0] for w in common_words]
print(common_words_text)

['redsox', 'dodgers', 'series', 'world', 'sox', 'red', 'worldseries', 'boston', 'go', 'win']


**Find patterns among how the 10 most common words are used in different cities**

***Using dataframe***

The total number of tweets located in Los Angeles:

In [53]:
tweets.filter("user.location IN ('Los Angeles', 'Los Angeles, CA')") \
    .count()

1748

Find patterns among how the 10 most common words are used in Los Angeles:

In [54]:
from pyspark.sql.functions import *
tweets.select("text") \
    .withColumn("text", lower(col('text'))) \
    .withColumn("text", split("text", " ")) \
    .withColumn("text", explode("text")) \
    .withColumn("text", regexp_replace("text", '[^a-z0-9]', '')) \
    .filter("user.location IN ('Los Angeles', 'Los Angeles, CA')") \
    .filter(col('text').isin(common_words_text)) \
    .groupBy("text") \
    .count() \
    .orderBy("count", ascending = False) \
    .show()

+-----------+-----+
|       text|count|
+-----------+-----+
|    dodgers| 1030|
|     series|  300|
|      world|  287|
|     redsox|  231|
|worldseries|  206|
|        sox|  199|
|        red|  190|
|         go|  101|
|        win|  101|
|     boston|   87|
+-----------+-----+



The total number of tweets located in Boston:

In [55]:
tweets.filter("user.location IN ('Boston', 'Boston, MA')") \
    .count()

3050

Find patterns among how the 10 most common words are used in Boston:

In [56]:
from pyspark.sql.functions import *
tweets.select("text") \
    .withColumn("text", lower(col('text'))) \
    .withColumn("text", split("text", " ")) \
    .withColumn("text", explode("text")) \
    .withColumn("text", regexp_replace("text", '[^a-z0-9]', '')) \
    .filter("user.location IN ('Boston', 'Boston, MA')") \
    .filter(col('text').isin(common_words_text)) \
    .groupBy("text") \
    .count() \
    .orderBy("count", ascending = False) \
    .show()

+-----------+-----+
|       text|count|
+-----------+-----+
|     redsox| 1700|
|     series|  709|
|      world|  680|
|        sox|  608|
|        red|  581|
|worldseries|  546|
|     boston|  318|
|    dodgers|  260|
|         go|  257|
|        win|  228|
+-----------+-----+



The total number of tweets located in Washington, DC:

In [57]:
tweets.filter("user.location IN ('Washington', 'Washington, DC')") \
    .count()

343

Find patterns among how the 10 most common words are used in Washington, DC:

In [58]:
from pyspark.sql.functions import *
tweets.select("text") \
    .withColumn("text", lower(col('text'))) \
    .withColumn("text", split("text", " ")) \
    .withColumn("text", explode("text")) \
    .withColumn("text", regexp_replace("text", '[^a-z0-9]', '')) \
    .filter("user.location IN ('Washington', 'Washington, DC')") \
    .filter(col('text').isin(common_words_text)) \
    .groupBy("text") \
    .count() \
    .orderBy("count", ascending = False) \
    .show()

+-----------+-----+
|       text|count|
+-----------+-----+
|     redsox|  149|
|      world|   94|
|     series|   93|
|        sox|   84|
|        red|   81|
|    dodgers|   45|
|     boston|   43|
|worldseries|   40|
|         go|   20|
|        win|   20|
+-----------+-----+



***Using SQL***

The total number of tweets located in Los Angeles:

In [59]:
sqlc.sql("""
    SELECT COUNT(*) AS count
    FROM tweets
    WHERE user.location IN ('Los Angeles', 'Los Angeles, CA')
""").show()

+-----+
|count|
+-----+
| 1748|
+-----+



In [60]:
sqlc.sql("""
    SELECT user.location
    FROM tweets
    WHERE user.location IN ('Los Angeles', 'Los Angeles, CA')
    GROUP BY user.location
""").show()

+---------------+
|       location|
+---------------+
|    Los Angeles|
|Los Angeles, CA|
+---------------+



Find patterns among how the 10 most common words are used in Los Angeles:

In [61]:
sqlc.sql("""
    SELECT word, count
    FROM (SELECT REGEXP_REPLACE(col, '[^a-z0-9]', '') AS word, COUNT(col) AS count
          FROM (SELECT EXPLODE(SPLIT(LOWER(text), ' '))
                FROM tweets
                WHERE user.location IN ('Los Angeles', 'Los Angeles, CA'))
          GROUP BY word)
    WHERE word IN ('redsox', 'dodgers', 'series', 'world', 'sox', 'red', 'worldseries', 'boston', 'go', 'win')
    ORDER BY count DESC
""").show()

+-----------+-----+
|       word|count|
+-----------+-----+
|    dodgers| 1030|
|     series|  300|
|      world|  287|
|     redsox|  231|
|worldseries|  206|
|        sox|  199|
|        red|  190|
|         go|  101|
|        win|  101|
|     boston|   87|
+-----------+-----+



Of the 1748 total tweets, 1030 mention dodgers...

provide similar proportions for red sox...

The total number of tweets located in Boston:

In [62]:
sqlc.sql("""
    SELECT COUNT(*) AS count
    FROM tweets
    WHERE user.location IN ('Boston', 'Boston, MA')
""").show()

+-----+
|count|
+-----+
| 3050|
+-----+



Find patterns among how the 10 most common words are used in Boston:

In [63]:
sqlc.sql("""
    SELECT word, count
    FROM (SELECT REGEXP_REPLACE(col, '[^a-z0-9]', '') AS word, COUNT(col) AS count
          FROM (SELECT EXPLODE(SPLIT(LOWER(text), ' '))
                FROM tweets
                WHERE user.location IN ('Boston', 'Boston, MA'))
          GROUP BY word)
    WHERE word IN ('redsox', 'dodgers', 'series', 'world', 'sox', 'red', 'worldseries', 'boston', 'go', 'win')
    ORDER BY count DESC
""").show()

+-----------+-----+
|       word|count|
+-----------+-----+
|     redsox| 1700|
|     series|  709|
|      world|  680|
|        sox|  608|
|        red|  581|
|worldseries|  546|
|     boston|  318|
|    dodgers|  260|
|         go|  257|
|        win|  228|
+-----------+-----+



The total number of tweets located in Washington, DC:

In [64]:
sqlc.sql("""
    SELECT COUNT(*) AS count
    FROM tweets
    WHERE user.location IN ('Washington', 'Washington, DC')
""").show()

+-----+
|count|
+-----+
|  343|
+-----+



Find patterns among how the 10 most common words are used in Washington, DC:

In [65]:
sqlc.sql("""
    SELECT word, count
    FROM (SELECT REGEXP_REPLACE(col, '[^a-z0-9]', '') AS word, COUNT(col) AS count
          FROM (SELECT EXPLODE(SPLIT(LOWER(text), ' '))
                FROM tweets
                WHERE user.location IN ('Washington', 'Washington, DC'))
          GROUP BY word)
    WHERE word IN ('redsox', 'dodgers', 'series', 'world', 'sox', 'red', 'worldseries', 'boston', 'go', 'win')
    ORDER BY count DESC
""").show()

+-----------+-----+
|       word|count|
+-----------+-----+
|     redsox|  149|
|      world|   94|
|     series|   93|
|        sox|   84|
|        red|   81|
|    dodgers|   45|
|     boston|   43|
|worldseries|   40|
|         go|   20|
|        win|   20|
+-----------+-----+



**Discussion**

For our analysis of the text portions of the tweets, we found that users tweeted more often in Boston compared to Los Angeles. This coincides with our previous theory that more tweets occurred in Boston because the Red Sox won the World Series in 5 games. 

We also noticed that dodgers was the most common word for tweets from LA and redsox was the most common word for tweets from Boston. Both cities also show similar proportions for referencing their teams within the text of their tweets. For example, nearly 59% of the tweets from LA specifically reference the Dodgers and the tweets from Boston reference the Red Sox specifically around 56% of the time. 

Finally, we provided a quick analysis of tweets from Washington DC during the same time period. As expected, this part of the analysis found that less tweets occurred in Washington DC when compared to LA and Boston. Also, the tweets from Washington DC reference the Red Sox more often than the Dodgers. So, the pattern of users tweeting more often about the winning team holds true and DC region may provide more of a local fan base for Boston since both cities are located on the East Coast of the country. 

**Disclaimer**

All members worked on the project individually from the top to the end and met together for the finding discussions. Everyone contributed substantially to the work.